# Session 5 — DDL, DML & Transactions (+ Subqueries)

Create/alter tables, modify data safely, and use subqueries for advanced filtering.


## Environment Setup

In [1]:
import sys, sqlite3, pandas as pd, numpy as np, matplotlib.pyplot as plt
print(sys.version)
import seaborn as sns
sns.set_theme()
from pathlib import Path
DB_PATH = Path('course.db')
conn = sqlite3.connect(DB_PATH)
conn.execute('PRAGMA foreign_keys=ON;')
print('SQLite ready at', DB_PATH.resolve())

3.11.9 (v3.11.9:de54cf5be3, Apr  2 2024, 07:12:50) [Clang 13.0.0 (clang-1300.0.29.30)]
SQLite ready at /Users/subhasht/Desktop/VSCode Programs/GenAI_Curriculum/Phase1/Data_Engineering/course.db


In [2]:
def run_sql(q, params=None):
    params = params or {}
    df = pd.read_sql_query(q, conn, params=params)
    display(df)
    return df

## 1. DDL — Alter Table

In [3]:
conn.execute("ALTER TABLE customers ADD COLUMN loyalty_tier TEXT DEFAULT 'Standard';");
conn.commit()
run_sql("SELECT * FROM customers;")

,customer_id,name,city,country,address,loyalty_tier
0,1,Aria,Austin,USA,1 River Rd,Standard
1,2,Ben,Berlin,Germany,Karlstr. 9,Standard
2,3,Chloe,Chicago,USA,Unknown,Standard
3,4,Dai,Denver,USA,11 Pine St,Standard


,customer_id,name,city,country,address,loyalty_tier
0,1,Aria,Austin,USA,1 River Rd,Standard
1,2,Ben,Berlin,Germany,Karlstr. 9,Standard
2,3,Chloe,Chicago,USA,Unknown,Standard
3,4,Dai,Denver,USA,11 Pine St,Standard


## 2. DML and Transactions

In [4]:
print("Before:")
run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")
try:
    conn.execute("BEGIN;")
    conn.execute("UPDATE order_items SET quantity = quantity + 1 WHERE order_id IN (1,2);")
    print("During (uncommitted):")
    run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")
    conn.execute("ROLLBACK;")
finally:
    pass
print("After rollback:")
run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")

Before:


,order_id,total_qty
0,1,5
1,2,1
2,3,2
3,4,1


During (uncommitted):


,order_id,total_qty
0,1,7
1,2,2
2,3,2
3,4,1


After rollback:


,order_id,total_qty
0,1,5
1,2,1
2,3,2
3,4,1


,order_id,total_qty
0,1,5
1,2,1
2,3,2
3,4,1


## 3. Subqueries Example

In [5]:
run_sql('''
SELECT o.order_id, COUNT(*) AS n_items
FROM order_items oi
JOIN orders o ON oi.order_id=o.order_id
GROUP BY o.order_id
HAVING n_items > (SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt FROM order_items GROUP BY order_id) t);
''')

,order_id,n_items
0,1,2


,order_id,n_items
0,1,2
